In [14]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import nltk
import boto3
from botocore.exceptions import ClientError
import os
import json
import pymorphy2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
import word2vec

In [55]:
twitter_data = pd.read_csv("Corona_NLP_train.csv", encoding = "ISO-8859-1")
twitter_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [56]:
list_of_all_tweets = list(twitter_data['OriginalTweet'])

In [70]:
print(list_of_all_tweets[0])

print(clean([list_of_all_tweets[0]]))

@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8
['and and']


Для начала немного преподготовим наши данные, и объеденим Extremely Negative с Negative  
и Extremely Positive с Positive

In [71]:
twitter_data.loc[twitter_data.Sentiment == 'Extremely Negative', 'Sentiment'] = 'Negative'
twitter_data.loc[twitter_data.Sentiment == 'Extremely Positive', 'Sentiment'] = 'Positive'

twitter_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Negative


Теперь очистим данные от хэштегов, ссылок и прочего

In [83]:
def clean(data):
    res = []
    for i in range(0, len(data)):
        temp = data[i]
        temp = re.sub("@\S+", " ", temp)
        temp = re.sub("https*\S+", " ", temp)
        temp = re.sub("#\S+", " ", temp)
        temp = re.sub("\'\w+", '', temp)
        temp = re.sub('[%s]' % re.escape(string.punctuation), ' ', temp)
        temp = re.sub(r'\w*\d+\w*', '', temp)
        temp = re.sub('\s{2,}', " ", temp)
        list_of_temp = temp.split(" ")
        out = []
        for temp in list_of_temp:
            if not temp.isalpha():
                continue
            if temp == '':
                continue
            else:
                out.append(temp.lower())
        temp = ' '.join(out)
        res.append(temp)
    return res

In [84]:
out = clean(list(twitter_data['OriginalTweet']))

Создадим новый датасет, с которым и будем работать в будущем.  
Для определения тональности текста нам не понадобятся username, ScreenName, Location, TweetAt

In [85]:
clean_twitts = pd.DataFrame()

clean_twitts['OriginalTweet'] = out
clean_twitts['Sentiment'] = twitter_data['Sentiment']

clean_twitts.head()

,OriginalTweet,Sentiment
0,and and,Neutral
1,advice talk to your neighbours family to excha...,Positive
2,coronavirus australia woolworths to give elder...,Positive
3,my food stock is not the only one which is emp...,Positive
4,me ready to go at supermarket during the outbr...,Negative


In [9]:
# Посмотрим какие классы у нас вообще есть
list_of_labels = list(clean_twitts['Sentiment'].unique())
list_of_labels

['Neutral', 'Positive', 'Negative']

In [10]:
# функция разделяет твитты по классам и записывает их в файл, нужно мне для анализа в amazon comprehend
def get_twitts(twitter_data, label, filename):
    l = twitter_data[twitter_data.Sentiment == label]
    l = list(l['OriginalTweet'])
    
    f = open(f"{filename}.txt", "w")
    for twitt in l[0:int(len(l)/10)]:
        f.write(twitt + "\n")
    f.close()

In [12]:
# создадим файл для каждого класса
# в результате у нас есть три файла
# в каждом файле-классе только твитты относящиеся к этому классу
for label in list_of_labels:
    get_twitts(clean_twitts, label, label)

Далее а Amazon Comprehend я для каждого файла сделала анализ на Sentiment, по строчно.  
Так как у меня в каждом файле хранятся твитты (разделенные новой строкой), относящиеся только к классу с названием файла, метрику accuracy можно посчитать следующим образом

In [11]:
def get_accuracy(filename, clean=False):
    jsons = []
    with open(filename) as f:
        for line in f:
            jsons.append(json.loads(line))
    
    score = 0
    for j in jsons:
        # сравниваю название файла с предсказаной тональностью (должны совпадать)
        original_sentiment = j['File'].split(".")[0].lower()
        if clean:
            original_sentiment = original_sentiment.split("_")[0]
        pred_sentiment = j['Sentiment'].lower()
        if original_sentiment == pred_sentiment:
            score += 1
                        
    return score / len(jsons) * 100

Как мы видим с помощью амазон сервиса мы получили крайне низкий результат accuracy  
Лишь немного выше чем рандомный выбор с вероятностью угадать 1/3

In [12]:
accuracy = get_accuracy("sentiment_from_comprehend.txt")
print(f"Accuracy = {accuracy}%")

Accuracy = 39.499270782693245%


Теперь попробуем всё тоже самое, только удалим стоп-слова и преобразуем все слова в начальную форму.  
Для этого создадим ещё один датафрейм, потому что тот нам еще понадобится

In [94]:
# преобразуем в начальную форму и удаляем стоп-слова

list_of_twitts = list(clean_twitts['OriginalTweet'])
out_list = []

morph = pymorphy2.MorphAnalyzer(lang='uk')

for twitt in list_of_twitts:
    # удаляем стоп-слова
    filtered_words = [word for word in twitt.split(" ") if word not in stopwords.words('english')]
    # преобразуем в начальную форму
    normal_form_words = []
    for word in filtered_words:
        normal_form_words.append(morph.parse(word)[0].normal_form)
    # объединяем
    out_list.append(' '.join(normal_form_words))

In [96]:
clean_and_normal = pd.DataFrame()

clean_and_normal['OriginalTweet'] = out_list
clean_and_normal['Sentiment'] = twitter_data['Sentiment']

clean_and_normal.head()

,OriginalTweet,Sentiment
0,,Neutral
1,advice talk neighbours family exchange phone n...,Positive
2,coronavirus australia woolworths give elderly ...,Positive
3,food stock one empty please panic enough food ...,Positive
4,ready go supermarket outbreak paranoid food st...,Negative


In [98]:
# создадим такие же файлы как и раньше
for label in list_of_labels:
    get_twitts(clean_and_normal, label, label + '_cleanup')

Далее всё тоже самое с Amazon Comprehend  
Как можно наблюдать точность уменьшилась, что меня несколько удивило

In [100]:
accuracy = get_accuracy("output", True)
print(f"Accuracy = {accuracy}%")

Accuracy = 31.186193485658727%


Теперь попробуем написать что-то своё

In [16]:
# нам нужна функция которая посчитает метрику accuracy схожим способом что и раньше
def get_accuracy_from_list(real, predictions):
    score = 0
    for i in range(len(real)):
        if real[i] == predictions[i]:
            score += 1
    return score / len(real) * 100

Далее поделим наши данные на тестовую и обучающие выборки, сначала для данных, очищенных только от хэштегов, ссылок и прочего

In [19]:
X_train, X_test, y_train, y_test = train_test_split(clean_twitts['OriginalTweet'], clean_twitts['Sentiment'], test_size=0.2, random_state=42)

Преобразуем данные в вектор

In [17]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000)

In [20]:
train_data_features = vectorizer.fit_transform(list(X_train)).toarray()

In [51]:
scores = cross_val_score(LogisticRegression(), train_data_features, y_train, cv=5)

Как можно наблюдать даже простая логистическая регрессия справилась с нашей задачей куда лучаше

In [52]:
print("Accuracy: {:.2f}".format(np.mean(scores)))

Accuracy: 0.79


Всё тоже самое, только для нормализованных данных

In [53]:
X_train, X_test, y_train, y_test = train_test_split(clean_and_normal['OriginalTweet'], clean_and_normal['Sentiment'], test_size=0.2, random_state=42)
train_data_features = vectorizer.fit_transform(list(X_train)).toarray()
scores = cross_val_score(LogisticRegression(), train_data_features, y_train, cv=5)

In [54]:
print("Accuracy: {:.2f}".format(np.mean(scores)))

Accuracy: 0.79


С RandomForest

In [55]:
train_data_features = vectorizer.fit_transform(list(X_train)).toarray()
scores = cross_val_score(RandomForestClassifier(n_estimators = 200), train_data_features, y_train, cv=5)
print("Accuracy: {:.2f}".format(np.mean(scores)))

Accuracy: 0.74


In [48]:
from gensim.models import Word2Vec

# Set values for various parameters
num_features = 5000    # Word vector dimensionality                      
min_word_count = 3    # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

model = Word2Vec(list(clean_and_normal['OriginalTweet']), workers=num_workers, min_count = min_word_count, window = context, sample = downsampling)

model_name = "clean_and_normail_data"
model.save(model_name)

In [55]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index_to_key)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [56]:
trainDataVecs = getAvgFeatureVecs( list(clean_and_normal), model, num_features )

Review 0 of 2


TypeError: 'Word2Vec' object is not subscriptable

In [86]:
most_frequent = dict()

for tweet in list(clean_twitts['OriginalTweet']):
    words = tweet.split(' ')
    for word in words:
        if most_frequent.get(word) is None:
            most_frequent[word] = 1
        else:
            most_frequent[word] += 1

In [87]:
len(most_frequent)

32184

In [101]:
sorted(most_frequent.items(), key=lambda x: x[1], reverse=True)

[('the', 44771),
 ('to', 38457),
 ('and', 24060),
 ('of', 21543),
 ('a', 19426),
 ('in', 19295),
 ('for', 14047),
 ('is', 12256),
 ('are', 11339),
 ('covid', 10545),
 ('i', 10026),
 ('you', 9705),
 ('on', 9432),
 ('this', 7970),
 ('at', 7842),
 ('prices', 7841),
 ('store', 6878),
 ('food', 6859),
 ('supermarket', 6692),
 ('we', 6648),
 ('with', 6631),
 ('it', 6612),
 ('that', 6494),
 ('have', 6157),
 ('as', 6101),
 ('grocery', 6094),
 ('be', 5745),
 ('people', 5547),
 ('from', 5299),
 ('amp', 5187),
 ('all', 4808),
 ('your', 4597),
 ('not', 4537),
 ('will', 4495),
 ('consumer', 4402),
 ('my', 4201),
 ('can', 4194),
 ('they', 4098),
 ('our', 4043),
 ('up', 3876),
 ('out', 3846),
 ('has', 3800),
 ('or', 3732),
 ('by', 3720),
 ('more', 3712),
 ('but', 3564),
 ('if', 3536),
 ('shopping', 3367),
 ('online', 3357),
 ('how', 3322),
 ('their', 3223),
 ('during', 3203),
 ('so', 3106),
 ('now', 3101),
 ('no', 2988),
 ('get', 2871),
 ('about', 2833),
 ('what', 2818),
 ('need', 2712),
 ('who', 270